# Country GDP ETL Pipeline
### A Professional Data Engineering Mini-Project

This repository implements a complete ETL (Extract, Transform, Load) workflow for global GDP data.
The project includes:
- A clean and well-documented Jupyter Notebook (this file)
- A standalone Python script (`etl.py`) to run the ETL from the command line
- A `requirements.txt` file for reproducible installs
- A GitHub Actions workflow to run basic checks on push
- A polished `README.md` aimed at recruiters and technical reviewers

---


In [ ]:
# Install requirements if running in a fresh environment (uncomment if needed)
# !pip install -r requirements.txt


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime
import logging
import sys

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

def log_progress(message):
    logger.info(message)


In [ ]:
log_progress("Starting data extraction...")

url = "https://www.worldometers.info/gdp/gdp-by-country/"
response = requests.get(url, timeout=15)

if response.status_code != 200:
    raise RuntimeError(f"Failed to retrieve data from {url} (status {response.status_code}).")

soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table")
if table is None:
    raise RuntimeError("Could not find the data table on the page.")

df = pd.read_html(str(table))[0]
log_progress(f"Extracted table with {len(df)} rows.")
df.head()


In [ ]:
log_progress("Starting data transformation...")

# Example of hygienic column cleaning
df.columns = [str(col).strip().replace('\n', ' ') for col in df.columns]

# Try to coerce GDP column to numeric (remove $, commas, footnotes)
gdp_col = None
for c in df.columns:
    if 'GDP' in c or 'Nominal' in c:
        gdp_col = c
        break

if gdp_col is None:
    raise RuntimeError("Could not detect a GDP column automatically.")

df[gdp_col] = (
    df[gdp_col]
    .astype(str)
    .str.replace('[\$,]', '', regex=True)
    .str.replace(',', '', regex=False)
    .str.extract(r'([0-9\.]+)', expand=False)
    .astype(float)
)

# Optional: rename important columns to recruiter-friendly names
df = df.rename(columns={gdp_col: 'GDP_USD_2024', 'Country (or dependent territory)': 'Country'})
df = df.loc[:, ~df.columns.duplicated()]  # drop duplicated columns if any

log_progress("Transformation finished.")
df.head()


In [ ]:
log_progress("Starting data load to SQLite...")

db_path = 'gdp_data.db'
conn = sqlite3.connect(db_path)
df.to_sql('country_gdp', conn, if_exists='replace', index=False)
conn.close()

log_progress(f"Data loaded to {db_path} (table: country_gdp).")

# Show final shape
print(df.shape)
